In [1]:
!pip install faiss-cpu
!pip install sentence-transformers

   ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
   -- ------------------------------------- 1.0/15.0 MB 7.2 MB/s eta 0:00:02
   ------ --------------------------------- 2.4/15.0 MB 6.7 MB/s eta 0:00:02
   ----------- ---------------------------- 4.2/15.0 MB 7.2 MB/s eta 0:00:02
   --------------- ------------------------ 5.8/15.0 MB 7.3 MB/s eta 0:00:02
   ------------------ --------------------- 7.1/15.0 MB 7.0 MB/s eta 0:00:02
   ---------------------- ----------------- 8.4/15.0 MB 7.0 MB/s eta 0:00:01
   -------------------------- ------------- 10.0/15.0 MB 7.1 MB/s eta 0:00:01
   ----------------------------- ---------- 11.0/15.0 MB 6.6 MB/s eta 0:00:01
   -------------------------------- ------- 12.3/15.0 MB 6.7 MB/s eta 0:00:01
   ------------------------------------- -- 13.9/15.0 MB 6.7 MB/s eta 0:00:01
   ---------------------------------------- 15.0/15.0 MB 6.6 MB/s eta 0:00:00
  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13 kB)
 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [3]:
df = pd.read_csv('sample_text.csv')
df.shape

(8, 2)

In [4]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [5]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer('all-mpnet-base-v2')
vectors = encoder.encode(df.text)

C:\Users\Gyanada\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Gyanada\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Gyanada\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggin

In [6]:
vectors.shape

(8, 768)

In [7]:
vectors

array([[-0.00247395,  0.0362672 , -0.05290459, ..., -0.09152358,
        -0.0397    , -0.04330491],
       [-0.03357266,  0.00980515, -0.03250129, ..., -0.05165466,
         0.02245886, -0.03156183],
       [-0.01865324, -0.04051318, -0.01235387, ...,  0.00610584,
        -0.07179647,  0.02773853],
       ...,
       [-0.00066458,  0.0425213 , -0.05645508, ...,  0.01315471,
        -0.03183569, -0.04357664],
       [-0.03317154,  0.03252458, -0.0248484 , ...,  0.01174421,
         0.05747124,  0.00571022],
       [-0.00166394,  0.00413825, -0.04597083, ...,  0.02008527,
         0.05656242, -0.00161596]], shape=(8, 768), dtype=float32)

In [8]:
dim = vectors.shape[1]

In [9]:
import faiss
index = faiss.IndexFlatL2(dim)
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001EDC5CBA0A0> >

In [10]:
index.add(vectors)

In [11]:
search_query = "I want to buy a polo t-shirt"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [12]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [13]:
# faiss.normalize_L2(svec)

In [16]:
distances, I = index.search(svec, k = 2)

In [17]:
I

array([[3, 2]])

In [18]:
df.loc[[3, 2]]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [19]:
search_query

'I want to buy a polo t-shirt'

In [20]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [21]:
I.tolist()

[[3, 2]]

In [22]:
row_indices = I.tolist()[0]
row_indices


[3, 2]

In [23]:
df.loc[row_indices]


,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [24]:
search_query


'I want to buy a polo t-shirt'